## Problem 2: Matching Group Schedules

### 1.1) Problem Statement:

Scheduler for 1 or more people to find a meeting time that works for everyone.



#### Sample


Sample Input:
```
person1_Schedule =[[ ‘7:00’, ’8:30’], [’12:00’, ’13:00’], [’16:00’, ’18:00’]]
person1_DailyAct = [‘9:00’, ’19:00’]
person2_Schedule = [[ ‘9:00’, ’10:30’], [’12:20’, ’14:30’], [’14:00’, ’15:00’], [’16:00’, ’17:00’ ]]
person2_DailyAct = [‘9:00’, ’18: 30’]
duration_of_meeting =30
```

Sample output:
```
[[’10:30’, ’12:00’], [’15:00’, ’16:00’], [’18:00’, ’18:30’]]
```

### 1.2) Solution Approach:

After thinking through this a few ways, I decided on a straightforward but not super efficient approach.

That is, Make an array representing every space of time available, in this case

`Minute 0 through minute 24*60`
   
Mark everything unavailable that is either before the Latest (max) start time, or after the Earliest (min) end time.
Then, loop through the students time slots, marking off each minute as unavailable.

At the end, loop through the availability array looking for blocks of time >= duration of meeting requested.

Side note: the strings need to be converted into numbers so we can do accurate comparisons

### 1.3) Pseudocode:
```

// returns "hr:mm" in total minutes, as hr*60+mm

function parse_time_str(time_str):
    time_segments = split time_str on ":"
    hours = time_segments[0] as int
    minutes = time_segments[1] as int
    return hours * 60 + minutes

function net_window(*avail_windows):
    set start = avail_windows[0][0]
    set end = avail_windows[0][1]
    
    for window in avail_windows:
        s = parse_time_str(window[0])
        e = parse_time_str(window[1])
        start = max(start, s)
        end = min(end, e)

    return [start, end]
    
    
function _solution2(*schedules, *avail_windows, meeting_dur):

    start, end = net_window(*avail_windows)

    resolution = 24 * 60
    set available to array[] with resolution # of elements
    
    // initally mark time out of windows as unavailable, the rest as open
    for time from 0 to resolution:
        if time >= start and time <= end:
            available[time] = 1
        else:
            available[time] = 0

    // mark times in schedules as unavailable
    for schedule in schedules:
        s = parse_time_str(schedule[0])
        e = parse_time_str(schedule[1])
        
        for minute in range(s,e):
            available[minute] = 0
            
    
    // unavail like => [1,1,1,1,1,0,0,0,1,1,0,0,0,1,1,1,1,0,0,0,0,0,1,1,1,1]
    //                            ^   ^
    //                            L   R
    
    
    slots = []
    L = start
    
    while L <= end - meeting_dur:    // no need to go farther
        if available[L]:
            R = L + 1
            while R <= end:
                if not available[R]:
                    R -= 1
                    break //return to outer if and while loop
                R += 1 

            if R - L >= meeting_dur:
                slots.append([L,R])
            L = R - 1
                
                          
        L += 1
                    
    
    return slots

```         

### 1.4) Efficiency Analysis

 - n = number of students
 - m = number of pre-scheduled items per student
 - k = length of pre-scheduled items in minutes

I expect O(n^2), checking with Step Count Method.

First, analyze each function
```
1  function parse_time_str(time_str):
  2      time_segments = split time_str on ":"
  3      hours = time_segments[0] as int
  4      minutes = time_segments[1] as int
  5, 6     return hours * 60 + minutes
```
so parse_time_str(1) is O(6) => O(1)

```
1  function net_window(*avail_windows):
  2     set start = 0
  3     set end = 24*60

  n  1  for window in avail_windows:
       2-7       s = parse_time_str(window[0])
       8-11      e = parse_time_str(window[1])
       12,13     start = max(start, s)
       14,15     end = min(end, e)

  4,5   return [start, end]
```
so net_window(n) = 5 + 15n => O(n)

Chunk by chunk of the main function

```  
0   function solution2(*schedules, *avail_windows, meeting_dur):
  4+15n   start, end = net_window(*avail_windows)
```
4+15n + ...


```
 1        resolution = 24 * 60
 2-1442   set available to array[] with resolution # of elements
 
 //1442 + 8640 steps max
 1442+
     1440 1 
          for time from 0 to resolution:
          1,3  if time >= start and time <= end:
          3       available[time] = 1
          4    else:
          5       available[time] = 0

```
=> 10087 + 15n steps => O(n)


Even though there could be n schedules, it doesn't make sense if there are many more appointments than the available minutes in the day. Thus, in cases where we use `n * k * basic steps` steps, it should may out around `n * 1439 * basic steps` (each student's schedule is completely full + some overlap).

We could also add some checks to stop when no appointment windows are feasible.

```
//
    n 0    for schedule in schedules:
       1-7       s = parse_time_str(schedule[0])
       7-13      e = parse_time_str(schedule[1])

       13 + k 1  for minute in range(s,e):
               1     available[minute] = 0

```

Assuming this chunk is ~  n\*1439*14 => 20160n => O(n)

So this puts the total so far at 10087 + 20175n => O(n) steps

Finally, the time window resolution is fixed, so we have a big calculation that is only O(0)

```
0    slots = []
1    L = start
    
 1439*     while L <= end - meeting_dur:    // no need to go farther
   0          if available[L]:
   1             R = L + 1
   1438*        while R <= end:
     0                 if not available[R]:
     1                     R -= 1
     2                     break //return to outer if and while loop
     3                 R += 1 

   2            if R - L >= meeting_dur:
   3                slots.append([L,R])
   4            L = R - 1

   5        L += 1


2       return slots
```

1439\*6 \* (1440\*4) + 3 = 41473443  => O(1)

And we conclude that the function is really O(n)

41473443 + 10087 + 20175n  => O(n)

### 1.5) Python Solution:

In [20]:
def parse_time(time_str):
    time_segments = time_str.split(":")
    print(time_segments)
    hr,min = int(time_segments[0]), int(time_segments[1])
    return hr*60 + min

In [5]:
def time_str(time_mins):
    hrs = time_mins // 60
    mins = time_mins % 60
    return f"{hrs:02d}:{mins:02d}"

In [29]:
def net_window(avail_windows):
    start = 0
    end = 24*60
    
    print(f'Start {start}, end: {end}')
    for window in avail_windows:
        s = parse_time(window[0])
        e = parse_time(window[1])
        start = max(start, s)
        end = min(end, e)

    return [start, end]

In [7]:
def window_map(window):
    return parse_time(window[0]), parse_time(window[1])

In [8]:
class Schedule:
    def __init__(self, schedule, avail_window):
        self.schedule_windows = [window_map(s) for s in schedule]
        self.avail_window = window_map(avail_window)
    def __str__(self):
        return "".join(["Schedule: ", str(self.schedule), " Window: ", str(self.avail_window[0]), str(self.avail_window[1])])

In [38]:

   
def solution2(schedules=[],avail_windows = [], appointment=30):

    start, end = net_window(avail_windows)

    resolution = 24 * 60
    available = []
    
    for time in range(0,resolution):
        if time >= start and time < end:
            available.append(1)
        else:
            available.append(0)

    print(f'Available hours: {available}')

    for schedule in schedules:
        s = parse_time(schedule[0])
        e = parse_time(schedule[1])
        
        for minute in range(s,e):
            available[minute] = 0
            
    
    slots = []
    L = start
    
    while L <= end - appointment:
        if available[L]:
            R = L + 1
            while R <= end:
                if not available[R]:
                    R -= 1
                    break
                R += 1 

            if R - L >= appointment:
                slots.append([L,R])
            L = R - 1
                
                          
        L += 1
                    
    
    return slots

In [16]:
# for printing the result nicely
def time_str(time_mins):
    hrs = time_mins // 60
    mins = time_mins % 60
    return f"{hrs:02d}:{mins:02d}"

#### Run test case:

In [37]:
''' 
Sample input 
'''
person1_Schedule =[[ '7:00', '8:30'],  ['12:00', '13:00'],  ['16:00', '18:00']] 
person1_DailyAct = ['9:00', '19:00'] 
 
person2_Schedule = [[ '9:00', '10:30'],  ['12:20', '14:30'],  ['14:00', '15:00'], ['16:00', '17:00' ]] 
person2_DailyAct = ['9:00', '18: 30'] 
duration_of_meeting =30 
 
# Sample output  
expected_out = [['10:30', '12:00'], ['15:00', '16:00'], ['18:00', '18:30']] 


print(solution2([person1_Schedule, person2_Schedule],[person1_DailyAct, person2_DailyAct],appointment=duration_of_meeting))

Start 0, end: 1440
['9', '00']
['19', '00']
['9', '00']
['18', ' 30']


AttributeError: 'list' object has no attribute 'split'